In [4]:
import numpy as np
import cv2
from tqdm import tqdm
from tkinter import Tk, filedialog

In [5]:
ESCAPE_KEY_ASCII = 27

def onChange(value):
    pass

def process_image(img_bgr, contrast_value, brightness_value, use_clahe):
    """
    Aplica contraste centrado em 128, brilho e (opcional) CLAHE apenas no canal L.
    """
    img_float = img_bgr.astype(np.float32)

    # Contraste centrado em 128 e brilho (domínio):
    contraste = contrast_value / 100.0
    ajustada = contraste * (img_float - 128) + 128 + brightness_value
    ajustada = np.clip(ajustada, 0, 255).astype(np.uint8)

    if not use_clahe:
        return ajustada

    # CLAHE no canal de luminosidade (LAB)
    lab = cv2.cvtColor(ajustada, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# === Selecionar a Imagem ===
Tk().withdraw()
file_path = filedialog.askopenfilename(
    title="Selecione uma imagem",
    filetypes=[("Imagens", "*.jpg;*.jpeg;*.png;*.bmp;*.tiff")]
)

if not file_path:
    print("Nenhuma imagem selecionada. Encerrando...")
    exit()

img = cv2.imread(file_path)
if img is None:
    print("Erro ao carregar a imagem. Encerrando...")
    exit()

copyimg = img.copy()

# === Criar janela principal ===
windowTitle = "Ajuste de Brilho e Contraste (Antes e Depois)"
cv2.namedWindow(windowTitle, cv2.WINDOW_NORMAL)

# Redimensiona a janela para caber imagem original + ajustada lado a lado + trackbars
h, w = img.shape[:2]
cv2.resizeWindow(windowTitle, w * 2, h + 80)

# Trackbars
cv2.createTrackbar("Contraste", windowTitle, 100, 200, onChange)  # 100 = 1.0x
cv2.createTrackbar("Brilho", windowTitle, 0, 200, onChange)       # brilho em [0..200]

# Estados
prev_contrast = cv2.getTrackbarPos("Contraste", windowTitle)
prev_brightness = cv2.getTrackbarPos("Brilho", windowTitle)
apply_clahe = True
force_update = True  # força o primeiro processamento

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.2
thickness = 2
color = (255, 255, 255)
shadow_color = (0, 0, 0)
y_pos = 40

while True:
    # Lê os sliders
    current_contrast = cv2.getTrackbarPos("Contraste", windowTitle)
    current_brightness = cv2.getTrackbarPos("Brilho", windowTitle)

    changed = (current_contrast != prev_contrast) or (current_brightness != prev_brightness)

    if changed or force_update:
        # Reprocessa imagem quando há mudança ou toggle/reset
        for _ in tqdm(range(1), desc="Aplicando ajustes", unit="passo"):
            copyimg = process_image(img, current_contrast, current_brightness, apply_clahe)

        prev_contrast = current_contrast
        prev_brightness = current_brightness
        force_update = False

    # Combina antes/depois
    combined = np.hstack((img, copyimg))

    # Rótulos ANTES/DEPOIS
    cv2.putText(combined, "ANTES", (30, y_pos), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(combined, "ANTES", (30, y_pos), font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.putText(combined, "DEPOIS", (w + 30, y_pos), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(combined, "DEPOIS", (w + 30, y_pos), font, font_scale, color, thickness, cv2.LINE_AA)

    # Indicador CLAHE no canto inferior direito
    clahe_status = "CLAHE: ON" if apply_clahe else "CLAHE: OFF"
    text_size, _ = cv2.getTextSize(clahe_status, font, font_scale, thickness)
    text_x = combined.shape[1] - text_size[0] - 20
    text_y = combined.shape[0] - 20
    cv2.putText(combined, clahe_status, (text_x, text_y), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(combined, clahe_status, (text_x, text_y), font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.imshow(windowTitle, combined)

    key = cv2.waitKey(1) & 0xFF
    if key == ESCAPE_KEY_ASCII:
        break
    elif key == ord('c'):  # Alterna CLAHE e força reprocessar
        apply_clahe = not apply_clahe
        force_update = True
        print(f"CLAHE {'ativado' if apply_clahe else 'desativado'}.")
    elif key == ord('r'):  # Reset dos sliders e força reprocessar
        cv2.setTrackbarPos("Contraste", windowTitle, 100)
        cv2.setTrackbarPos("Brilho", windowTitle, 0)
        prev_contrast = 100
        prev_brightness = 0
        force_update = True
        print("🔄 Brilho e contraste resetados para os valores padrão.")

cv2.destroyAllWindows()

# === Salvar ===
Tk().withdraw()
save_path = filedialog.asksaveasfilename(
    defaultextension=".jpg",
    filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png"), ("BMP", "*.bmp"), ("TIFF", "*.tiff")],
    title="Salvar imagem editada como..."
)

if save_path:
    cv2.imwrite(save_path, copyimg)
    print(f"✅ Imagem salva em: {save_path}")
else:
    print("❌ Imagem não salva.")


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 104.63passo/s]


CLAHE desativado.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 141.89passo/s]


CLAHE ativado.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 104.72passo/s]


...

CLAHE ativado.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 107.33passo/s]


🔄 Brilho e contraste resetados para os valores padrão.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 117.61passo/s]


✅ Imagem salva em: C:/Users/Victor Macedo/Desktop/aaaa.jpg
